In [428]:
#Load the packages
import numpy as np
import pandas as pd
from numpy.linalg import inv
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import black_litterman
from pypfopt import expected_returns

## Question 1

Part A: Solve the mean variance optimization problem and replicate column 3 from the other example


In [429]:
#Import data
equities= pd.read_csv('he_litterman_data.csv', skiprows=1, nrows=7)[["Country", 
                                                "Equity Index Volatility (%)", 
                                                "Equilibrium Portfolio Weight (%)"]]
covariance_matrix= pd.read_csv('he_litterman_data.csv', skiprows=11, nrows=18)

In [430]:
# Solve for the equilibrium premium
cweights = np.array(equities["Equilibrium Portfolio Weight (%)"]/100)
cov_matrix = np.asmatrix(covariance_matrix.drop(columns=["Unnamed: 0"]))
equities["Equilibrium returns(%)"] = round(pd.DataFrame(np.transpose(np.dot(cov_matrix, weights)*2.5)),1)
prior_returns = np.array(equities["Equilibrium returns(%)"]/100)
equities

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium returns(%)
0,Australia,16.0,1.6,3.9
1,Canada,20.3,2.2,6.9
2,France,24.8,5.2,8.4
3,Germany,27.1,5.5,9.0
4,Japan,21.0,11.6,4.3
5,UK,20.0,12.4,6.8
6,USA,18.7,61.5,7.6


## Question 2

1: Formulate a view(any investor views)

In [431]:
# View: I have a relative view that Germany will outperform the rest of Europe (France and UK) by 5%
Q = np.array([0.05])
P = np.array([0, 0, 0, 0, 0, 0, 0],) 
P_dash = np.array([0, 0, -0.3, 1, 0, -0.7, 0],)
View_variance =  0.2**2

#I am assuming further that my view has a variance of 0.2**2

2: Specify the uncertainty with the CAPM 

In [432]:
# My uncertainity about the CAPM is 1
CAPM_uncertainty =  1
cov_matrix = cov_matrix * CAPM_uncertainty

3: Calculate the Black-Litterman risk premia and the new portifolio weight

In [433]:

#The new expected return is;
risk_aversion = 2.5
equities["BL risk premium(%)"]  = round(pd.DataFrame(np.dot(np.linalg.inv(np.linalg.inv(CAPM_uncertainty*cov_matrix) + 
            (P_dash*(1/View_variance)*P)) , 
            np.transpose(np.array(np.linalg.inv(CAPM_uncertainty*cov_matrix).dot(prior_returns)) + 
            (P_dash*(1/View_variance)*Q))))*100,1)
BL_premium = np.array(equities["BL risk premium(%)"])

#Solve for the new portfolio weight
#Get a discounting factor
disc_factor = np.linalg.inv(risk_aversion*cov_matrix)

#Get the actual number
equities["BL weights(%)"] = round(pd.DataFrame(np.transpose(disc_factor.dot(BL_premium))),1)
equities

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium returns(%),BL risk premium(%),BL weights(%)
0,Australia,16.0,1.6,3.9,4.5,-0.2
1,Canada,20.3,2.2,6.9,8.0,2.5
2,France,24.8,5.2,8.4,9.9,-9.2
3,Germany,27.1,5.5,9.0,12.3,54.4
4,Japan,21.0,11.6,4.3,4.6,11.6
5,UK,20.0,12.4,6.8,7.1,-21.2
6,USA,18.7,61.5,7.6,8.4,61.5


# Question 3: Check the effect of a change in the Uncertainty of the CAPM and Views variance



1. Lets explore the effect of change in uncertainty

In [436]:
#Lets try different values of CAPM uncertainity
CAPM_uncertainty = 0.01
equities["BL premium_0.5"]  = round(pd.DataFrame(np.dot(np.linalg.inv(np.linalg.inv(CAPM_uncertainty*cov_matrix) + 
            (P_dash*(1/View_variance)*P)) , 
            np.transpose(np.array(np.linalg.inv(CAPM_uncertainty*cov_matrix).dot(prior_returns)) + 
            (P_dash*(1/View_variance)*Q))))*100,1)
BL_premium = np.array(equities["BL premium_0.5"])

#Solve for the new portfolio weight
#Get a discounting factor
disc_factor = np.linalg.inv(risk_aversion*cov_matrix)

#Get the actual number
equities["BL weights_0.5"] = round(pd.DataFrame(np.transpose(disc_factor.dot(BL_premium))),1)


CAPM_uncertainty = 1.5
equities["BL premium_1.5"]  = round(pd.DataFrame(np.dot(np.linalg.inv(np.linalg.inv(CAPM_uncertainty*cov_matrix) + 
            (P_dash*(1/View_variance)*P)) , 
            np.transpose(np.array(np.linalg.inv(CAPM_uncertainty*cov_matrix).dot(prior_returns)) + 
            (P_dash*(1/View_variance)*Q))))*100,1)
BL_premium = np.array(equities["BL premium_1.5"])

#Solve for the new portfolio weight
#Get a discounting factor
disc_factor = np.linalg.inv(risk_aversion*cov_matrix)

#Get the actual number
equities["BL weights_1.5"] = round(pd.DataFrame(np.transpose(disc_factor.dot(BL_premium))),1)

equities

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium returns(%),BL risk premium(%),BL weights(%),BL premium_0.5,BL weights_0.5,BL premium_1.5,BL weights_1.5,BL premium_varL,BL weights_varL,BL premium_varH,BL weights_varH
0,Australia,16.0,1.6,3.9,4.5,-0.2,3.9,0.6,4.1,0.7,7.1,0.6,4.0,-0.6
1,Canada,20.3,2.2,6.9,8.0,2.5,6.9,1.1,7.2,-0.6,12.3,0.2,7.1,0.1
2,France,24.8,5.2,8.4,9.9,-9.2,8.4,6.4,8.9,3.0,16.1,-67.6,8.7,2.2
3,Germany,27.1,5.5,9.0,12.3,54.4,9.0,4.2,10.0,18.6,25.6,254.2,9.7,15.0
4,Japan,21.0,11.6,4.3,4.6,11.6,4.3,11.7,4.4,11.7,6.0,12.3,4.4,12.2
5,UK,20.0,12.4,6.8,7.1,-21.2,6.8,13.1,6.9,2.2,8.3,-163.4,6.9,6.9
6,USA,18.7,61.5,7.6,8.4,61.5,7.6,62.9,7.9,64.8,11.8,63.6,7.8,64.1


After trying different values of the CAPM uncertainity it appears that as the uncertainity in the CAPM decreases, the risk premium decreases and so investors will want to be compesanted less for the risk. 


In [435]:
#Lets try different values of Views variance
CAPM_uncertainty = 1
View_variance = 0.2**3
equities["BL premium_varL"]  = round(pd.DataFrame(np.dot(np.linalg.inv(np.linalg.inv(CAPM_uncertainty*cov_matrix) + 
            (P_dash*(1/View_variance)*P)) , 
            np.transpose(np.array(np.linalg.inv(CAPM_uncertainty*cov_matrix).dot(prior_returns)) + 
            (P_dash*(1/View_variance)*Q))))*100,1)
BL_premium = np.array(equities["BL premium_varL"])

#Solve for the new portfolio weight
#Get a discounting factor
disc_factor = np.linalg.inv(risk_aversion*cov_matrix)

#Get the actual number
equities["BL weights_varL"] = round(pd.DataFrame(np.transpose(disc_factor.dot(BL_premium))),1)


View_variance = 0.2
equities["BL premium_varH"]  = round(pd.DataFrame(np.dot(np.linalg.inv(np.linalg.inv(CAPM_uncertainty*cov_matrix) + 
            (P_dash*(1/View_variance)*P)) , 
            np.transpose(np.array(np.linalg.inv(CAPM_uncertainty*cov_matrix).dot(prior_returns)) + 
            (P_dash*(1/View_variance)*Q))))*100,1)
BL_premium = np.array(equities["BL premium_varH"])

#Solve for the new portfolio weight
#Get a discounting factor
disc_factor = np.linalg.inv(risk_aversion*cov_matrix)

#Get the actual number
equities["BL weights_varH"] = round(pd.DataFrame(np.transpose(disc_factor.dot(BL_premium))),1)

equities

,Country,Equity Index Volatility (%),Equilibrium Portfolio Weight (%),Equilibrium returns(%),BL risk premium(%),BL weights(%),BL premium_0.5,BL weights_0.5,BL premium_1.5,BL weights_1.5,BL premium_varL,BL weights_varL,BL premium_varH,BL weights_varH
0,Australia,16.0,1.6,3.9,4.5,-0.2,4.2,-0.2,4.9,1.3,7.1,0.6,4.0,-0.6
1,Canada,20.3,2.2,6.9,8.0,2.5,7.4,0.1,8.5,-0.5,12.3,0.2,7.1,0.1
2,France,24.8,5.2,8.4,9.9,-9.2,9.2,-1.0,10.7,-16.8,16.1,-67.6,8.7,2.2
3,Germany,27.1,5.5,9.0,12.3,54.4,10.7,29.5,14.0,79.4,25.6,254.2,9.7,15.0
4,Japan,21.0,11.6,4.3,4.6,11.6,4.5,12.1,4.8,11.5,6.0,12.3,4.4,12.2
5,UK,20.0,12.4,6.8,7.1,-21.2,7.0,-3.4,7.3,-38.6,8.3,-163.4,6.9,6.9
6,USA,18.7,61.5,7.6,8.4,61.5,8.0,62.6,8.9,64.4,11.8,63.6,7.8,64.1


Same for the change in the views confidence, the less the confidence, the higher the premium to all asset classes and specifically high to countries are directly linked with the view in a positive way. The increase in confidence reduces the risk premium and this suggests a less compesation required by investors 